In [1]:
import pandas as pd

In [4]:
path_to_mpa_output="/shared/projects/mudis4ls_is4_benchmark/test_results/metaphlan/metaphlan_output_gtdb.txt"
path_to_sylph_output="/shared/projects/mudis4ls_is4_benchmark/test_results/sylph/ERS12377136.fastq.gz.sylphmpa"
path_to_motus_output="/shared/projects/mudis4ls_is4_benchmark/test_results/motus/results.txt"
path_to_bracken_output="/shared/projects/mudis4ls_is4_benchmark/test_results/kraken/gtdb/test_Species.bracken"

In [3]:
# Charger les fichiers en DataFrames
df_metaphlan = pd.read_csv(path_to_mpa_output, sep="\t", skiprows=1 )
df_sylph = pd.read_csv(path_to_sylph_output, sep="\t", skiprows=1)
df_motus= pd.read_csv(path_to_motus_output, sep="\t")

In [5]:
df_bracken=pd.read_csv(path_to_bracken_output, sep="\t")

In [6]:
df_motus

,#mOTUs2_clade,ERS12377136
0,k__Bacteria,0.731786
1,k__Bacteria|p__Proteobacteria,0.044189
2,k__Bacteria|p__Firmicutes,0.401600
3,k__Bacteria|p__Verrucomicrobia,0.019538
4,k__Bacteria|p__Bacteroidetes,0.266460
...,...,...
85,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clo...,0.027161
86,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.005656
87,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.005656
88,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.003111


In [9]:
df_motus = df_motus.rename(columns={"ERS12377136": "abundance_motus"})

In [10]:
df_bracken

,name,taxonomy_id,taxonomy_lvl,kraken_assigned_reads,added_reads,new_est_reads,fraction_total_reads
0,Bacteroides stercoris,46506,S,3744,2,3746,0.07276
1,Bacteroides uniformis,820,S,2991,0,2991,0.05809
2,Bacteroides xylanisolvens,371601,S,1448,0,1448,0.02812
3,Bacteroides finegoldii,338188,S,994,0,994,0.01931
4,Bacteroides ovatus,28116,S,794,0,794,0.01542
...,...,...,...,...,...,...,...
1077,Methanolobus mangrovi,3072977,S,1,0,1,0.00002
1078,Homo sapiens,9606,S,5,0,5,0.00010
1079,Junduvirus communis,2956034,S,10,0,10,0.00019
1080,Junduvirus copri,2956035,S,1,0,1,0.00002


In [11]:
# Modifier le nom de la première colonne
df_metaphlan = df_metaphlan.rename(columns={df_metaphlan.columns[0]: df_metaphlan.columns[0][1:]})

print(df_metaphlan.head())

                      clade_name  relative_abundance
0                    d__Bacteria           100.00000
1    d__Bacteria;p__Bacteroidota            52.20608
2    d__Bacteria;p__Firmicutes_A            43.38615
3    d__Bacteria;p__Firmicutes_C             3.35772
4  d__Bacteria;p__Proteobacteria             1.05005


In [12]:
# Renommer la colonne relative_abundance pour chaque outil
df_metaphlan.rename(columns={"relative_abundance": "abundance_metaphlan"}, inplace=True)
df_sylph.rename(columns={"relative_abundance": "abundance_sylph"}, inplace=True)


In [13]:
# Remplacement efficace avec regex
df_metaphlan["clade_name"] = df_metaphlan["clade_name"].str.replace(
    r"Firmicutes", "Bacillota", regex=True
).str.replace(
    r"Proteobacteria", "Pseudomonadota", regex=True
)

In [14]:
df_metaphlan["clade_name"] = df_metaphlan["clade_name"].str.replace(";", "|")


In [15]:
df_merged = df_metaphlan.merge(df_sylph, on="clade_name", how="outer")


In [16]:
df_merged

,clade_name,abundance_metaphlan,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level)
0,d__Bacteria,100.00000,100.0000,100.0001,NaN,NaN
1,d__Bacteria|p__Bacillota_A,43.38615,41.4963,36.1829,NaN,NaN
2,d__Bacteria|p__Bacillota_A|c__Clostridia,43.38615,41.4963,36.1829,NaN,NaN
3,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN
4,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN
...,...,...,...,...,...,...
96,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
97,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
98,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
99,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN


In [18]:
df_motus.rename(columns={"#mOTUs2_clade": "clade_name"}, inplace=True)


In [19]:
df_motus

,clade_name,abundance_motus
0,k__Bacteria,0.731786
1,k__Bacteria|p__Proteobacteria,0.044189
2,k__Bacteria|p__Firmicutes,0.401600
3,k__Bacteria|p__Verrucomicrobia,0.019538
4,k__Bacteria|p__Bacteroidetes,0.266460
...,...,...
85,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clo...,0.027161
86,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.005656
87,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.005656
88,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.003111


In [20]:
# Remplacement efficace avec regex
df_motus["clade_name"] = df_motus["clade_name"].str.replace(
    r"Firmicutes", "Bacillota", regex=True
).str.replace(
    r"Proteobacteria", "Pseudomonadota", regex=True
)

In [21]:
# Remplacement efficace avec regex
df_motus["clade_name"] = df_motus["clade_name"].str.replace(
    r"k__", "d__", regex=True
)

In [22]:
df_motus

,clade_name,abundance_motus
0,d__Bacteria,0.731786
1,d__Bacteria|p__Pseudomonadota,0.044189
2,d__Bacteria|p__Bacillota,0.401600
3,d__Bacteria|p__Verrucomicrobia,0.019538
4,d__Bacteria|p__Bacteroidetes,0.266460
...,...,...
85,d__Bacteria|p__Bacillota|c__Clostridia|o__Clos...,0.027161
86,d__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.005656
87,d__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.005656
88,d__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...,0.003111


In [23]:
df_merged

,clade_name,abundance_metaphlan,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level)
0,d__Bacteria,100.00000,100.0000,100.0001,NaN,NaN
1,d__Bacteria|p__Bacillota_A,43.38615,41.4963,36.1829,NaN,NaN
2,d__Bacteria|p__Bacillota_A|c__Clostridia,43.38615,41.4963,36.1829,NaN,NaN
3,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN
4,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN
...,...,...,...,...,...,...
96,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
97,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
98,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
99,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN


In [24]:
df_merged = df_merged.merge(df_motus, on="clade_name", how="outer")


In [25]:
df_merged

,clade_name,abundance_metaphlan,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level),abundance_motus
0,d__Bacteria,100.00000,100.0000,100.0001,NaN,NaN,0.731786
1,d__Bacteria|p__Bacillota,NaN,NaN,NaN,NaN,NaN,0.401600
2,d__Bacteria|p__Bacillota_A,43.38615,41.4963,36.1829,NaN,NaN,NaN
3,d__Bacteria|p__Bacillota_A|c__Clostridia,43.38615,41.4963,36.1829,NaN,NaN,NaN
4,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN,NaN
...,...,...,...,...,...,...,...
183,d__Bacteria|p__Verrucomicrobia|c__Verrucomicro...,NaN,NaN,NaN,NaN,NaN,0.019538
184,d__Bacteria|p__Verrucomicrobia|c__Verrucomicro...,NaN,NaN,NaN,NaN,NaN,0.019538
185,d__Bacteria|p__Verrucomicrobia|c__Verrucomicro...,NaN,NaN,NaN,NaN,NaN,0.019538
186,d__Bacteria|p__Verrucomicrobia|c__Verrucomicro...,NaN,NaN,NaN,NaN,NaN,0.019538


In [14]:
output_dir='/shared/projects/mudis4ls_is4_benchmark/test_results/compare_mpa_sylph/'
df_merged.to_csv(output_dir+'compare_mpa_sylph.tsv',sep='\t',index=False)


In [15]:

# Species	
# Genus	
# Family	
# Order	
# Class
# Domain : bacteria

In [16]:
clade=df_merged["clade_name"]


In [17]:
clade

0                                            d__Bacteria
1                             d__Bacteria|p__Bacillota_A
2               d__Bacteria|p__Bacillota_A|c__Clostridia
3      d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
4      d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
                             ...                        
96     d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
97     d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
98     d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
99     d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
100    d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
Name: clade_name, Length: 101, dtype: object

In [18]:
# Assurez-vous que la fonction renvoie un tuple ou une liste pour chaque ligne
def get_last_taxonomic_level(clade):
    parts = clade.split('|')
    last = parts[-1] if parts else None
    print(last.startswith("d__"), last)  # Pour déboguer et voir ce qui est retourné
    if last and last.startswith("d__"):
        return "domain", last
    elif last and last.startswith("p__"):
        return "phylum", last
    elif last and last.startswith("c__"):
        return "class", last
    elif last and last.startswith("o__"):
        return "order", last
    elif last and last.startswith("f__"):
        return "family", last
    elif last and last.startswith("g__"):
        return "genus", last
    elif last and last.startswith("s__"):
        return "species", last
    else:
        return "unknown", last

# Appliquer la fonction à la colonne "clade_name" et obtenir deux colonnes
df_merged[['taxonomic_level', 'last_taxon']] = df_merged['clade_name'].apply(
    lambda x: pd.Series(get_last_taxonomic_level(x))
)

# Vérifiez si les colonnes sont remplies correctement
print(df_merged[['taxonomic_level', 'last_taxon']].head())


True d__Bacteria
False p__Bacillota_A
False c__Clostridia
False o__Lachnospirales
False f__Lachnospiraceae
False g__Agathobacter
False s__Agathobacter rectalis
False t__GCF_000020605.1
False g__Blautia
False g__Blautia_A
False s__Blautia_A wexlerae
False t__GCF_025148125.1
False s__Blautia hansenii
False t__GCF_002222595.2
False g__Brotaphodocola
False s__Brotaphodocola sp900540335
False t__GCA_900540335.1
False g__Enterocloster
False s__Enterocloster sp001517625
False t__GCF_001517625.2
False g__Lachnospira
False s__Lachnospira sp000437735
False t__GCF_020564355.1
False g__Mediterraneibacter
False s__Mediterraneibacter faecis
False s__Mediterraneibacter torques
False t__GCF_000153925.1
False g__Roseburia
False s__Roseburia intestinalis
False t__GCF_900537995.1
False g__Ventrimonas
False s__Ventrimonas sp900540335
False o__Oscillospirales
False f__Acutalibacteraceae
False g__Fimivicinus
False s__Fimivicinus sp900544715
False t__GCA_900544715.1
False g__Ruminococcus_E
False s__Ruminococ

/tmp/ipykernel_2010048/3983686473.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged[['taxonomic_level', 'last_taxon']] = df_merged['clade_name'].apply(
/tmp/ipykernel_2010048/3983686473.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged[['taxonomic_level', 'last_taxon']] = df_merged['clade_name'].apply(


In [19]:
df_merged

,clade_name,abundance_metaphlan,abundance_sylph,diff_abundance,taxonomic_level,last_taxon
0,d__Bacteria,100.00000,100.0000,0.00000,domain,d__Bacteria
1,d__Bacteria|p__Bacillota_A,43.38615,41.4963,1.88985,phylum,p__Bacillota_A
2,d__Bacteria|p__Bacillota_A|c__Clostridia,43.38615,41.4963,1.88985,class,c__Clostridia
3,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,-0.90287,order,o__Lachnospirales
4,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,-0.90287,family,f__Lachnospiraceae
...,...,...,...,...,...,...
96,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN,order,o__Burkholderiales
97,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN,family,f__Burkholderiaceae
98,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN,genus,g__Sutterella
99,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN,species,s__Sutterella wadsworthensis_A


In [63]:
output_dir='/shared/projects/mudis4ls_is4_benchmark/test_results/compare_mpa_sylph/'
df_merged.to_csv(output_dir+'compare_mpa_sylph.tsv',sep='\t',index=False)


In [20]:
# Calculer la moyenne de diff_abundance pour chaque taxonomic_level
mean_diff_abundance = df_merged.groupby("taxonomic_level")["diff_abundance"].mean()

# Afficher le résultat
print(mean_diff_abundance)


taxonomic_level
class      3.281000e-01
domain     0.000000e+00
family     1.640500e-01
genus      4.320700e-01
order      2.624800e-01
phylum    -3.108624e-15
species    1.156955e+00
unknown             NaN
Name: diff_abundance, dtype: float64


In [11]:
#Pseudomonadota – anciennement Proteobacteria 
#Bacillota – anciennement Firmicutes
# Regarde ce qui se passe si on change juste l'assignation de ces 2 clades

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
sns.heatmap(df.set_index("clade_name"), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Comparaison des abondances par clade et outil")
plt.show()
